In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from scipy import optimize

In [ ]:
# define function for performing kubic harmonic regression
def kubic_harm(prop,norm): #order is in (111), (100), and (110)
    x1 = [0.333333333,1,0.5]
    x2 = [0.037037037,0,0]
    fit_x = np.array([x1,x2]).transpose()
    model = LinearRegression()
    model.fit(fit_x,[prop[ii]/norm[ii] for ii in range(0,len(prop))])
    return [model.intercept_,model.coef_[0],model.coef_[1]]

In [ ]:
# define function for calculating equilibrium coverages over NP model with kubic harmonics regression
#### NOTE: MODEL CURRENTLY SETUP FOR H*/PT ####
def NP_calc(npoints,T,P_H2,kub_A,kub_B,kub_C,kub_mA,kub_mB,kub_mC,kub_bA,kub_bB,kub_bC):
    phi = np.radians(np.linspace(0,360,npoints))
    theta = np.radians(np.linspace(0,90,npoints))
    
    kB = 1.38064852*10**(-23) # J/K, Boltzmann Constant
    h = 6.62607004*10**(-34) # J*s, Plank's Constant
    mH2 = 2.016/(6.022*10**23)/1000 # kg, H2 Mass
    area_s = 27.57442205*(10**-10)**2/4 # m^2, area of the smallest facet (111)
    kB2 = 8.617333262145*10**(-5) # eV/K, Boltzmann Constant
    vH2 = 537.459121/1000 # eV, H2 vibrational mode
    redmassH2 = mH2/4 # kg, H2 reduced mass
    R_H2 = 0.74750*10**(-10) # m, H2 bond length
    I_H2 = redmassH2*R_H2**2 # kg*m^2, H2 moment of inertia
    sigma = 2 # unitless, symmetry number
    lamb = h/(2*np.pi*mH2*kB*T)**(1/2)
    H2_qvib = np.exp(-vH2/(2*kB2*T))/(1 - np.exp(-vH2/(kB2*T))) # H2 vibrational partition function
    H2_qrot = (8*np.pi**2*I_H2*kB*T)/(sigma*h**2) # H2 rotational partition function
    Zint = H2_qvib*H2_qrot
    H2_qtrans = 1/lamb**3*kB*T/P_H2 # H2 translational partition function
    
    covH = []
    for ii in range(0,npoints): # loop over all theta values
        for jj in range(0,npoints): # loop over all phi values        
            nx = np.sin(theta[ii])*np.cos(phi[jj])
            ny = np.sin(theta[ii])*np.sin(phi[jj])
            nz = np.cos(theta[ii])
            
            # calculate Eads vs coverage slope and intercept at point on NP
            Eads_A = kub_A[0] + kub_A[1]*(nx**4+ny**4+nz**4) + kub_A[2]*(nx**2*ny**2*nz**2)
            Eads_B = kub_B[0] + kub_B[1]*(nx**4+ny**4+nz**4) + kub_B[2]*(nx**2*ny**2*nz**2)
            Eads_C = kub_C[0] + kub_C[1]*(nx**4+ny**4+nz**4) + kub_C[2]*(nx**2*ny**2*nz**2)
            
            # calculate the best fit slope and intercept for Gvib(O/S) with coverage at point on NP
            Avib_mA = kub_mA[0] + kub_mA[1]*(nx**4+ny**4+nz**4) + kub_mA[2]*(nx**2*ny**2*nz**2)
            Avib_mB = kub_mB[0] + kub_mB[1]*(nx**4+ny**4+nz**4) + kub_mB[2]*(nx**2*ny**2*nz**2)
            Avib_mC = kub_mC[0] + kub_mC[1]*(nx**4+ny**4+nz**4) + kub_mC[2]*(nx**2*ny**2*nz**2)
            
            Avib_bA = kub_bA[0] + kub_bA[1]*(nx**4+ny**4+nz**4) + kub_bA[2]*(nx**2*ny**2*nz**2)
            Avib_bB = kub_bB[0] + kub_bB[1]*(nx**4+ny**4+nz**4) + kub_bB[2]*(nx**2*ny**2*nz**2)
            Avib_bC = kub_bC[0] + kub_bC[1]*(nx**4+ny**4+nz**4) + kub_bC[2]*(nx**2*ny**2*nz**2)
            
            # find equilibrium coverage at point on NP
            def func(x):
                Eads = Eads_A*x**2 + Eads_B*x + Eads_C
                Avib_T = (Avib_mA*x+Avib_bA)*T**2 + (Avib_mB*x+Avib_bB)*T + (Avib_mC*x+Avib_bC)
                Nsites = 16
                Gads = (Nsites*x*Eads + Avib_T - Nsites*x/2*(-kB2*T*np.log(Zint*H2_qtrans)))/Nsites
                Ke = np.exp(-Gads/kB2/T)
                fx = (Ke*P_H2 - 1)*x**2 - 2*Ke*P_H2*x + Ke*P_H2
                return fx

            xnew = optimize.brentq(func,0,1)
            covH.append(xnew)
    return [covH]

In [ ]:
#### NOTE: MODEL CURRENTLY SETUP FOR H*/PT ####
npoints = 300
T = 100 + 273.15 # K, Temperature
P_H2 = np.logspace(-12,2,15)*10**5 # Pa, H2 Pressure
phi = np.radians(np.linspace(0,360,npoints))
ph1,ph2 = np.meshgrid(phi,phi)

# normalization constants based on facet surfae area relative to (111), order is in [(111), (100), (110)]
norm_cov = [1.0,0.8831225935,0.6244625662]

# mean-field parabolic parameters of Eads versus normalized coverage, order is in [(111), (100), (110)]
A = [0.147,0.114,0.099]
B = [-0.062,-0.101,-0.018]
C = [-0.420,-0.551,-0.562]

# multi-variable regression parameters for Gvib versus normalized coverage and T, order is in [(111), (100), (110)]
mA = [-2.284420e-06,-2.260778e-06,-2.422571e-06]
mB = [2.917589e-04,7.204085e-04,-5.372658e-04]
mC = [3.088247,2.668010,3.014084]

bA = [-2.583573e-08,-4.180143e-09,-3.676194e-09]
bB = [-1.180756e-04,-1.455011e-05,2.313128e-05]
bC = [-6.232575e-04,-6.509515e-03,-7.953787e-03]

# fit regression parameters to kubic harmonics for Eads and Gvib
# Eads
# kub_A = kubic_harm(A,[x**2 for x in norm_cov])
kub_B = kubic_harm(B,norm_cov)
kub_C = kubic_harm(C,[1,1,1])

# Gvib
kub_mA = kubic_harm(mA,norm_cov)
kub_mB = kubic_harm(mB,norm_cov)
kub_mC = kubic_harm(mC,norm_cov)

kub_bA = kubic_harm(bA,[1,1,1])
kub_bB = kubic_harm(bB,[1,1,1])
kub_bC = kubic_harm(bC,[1,1,1])

# run monometallic NP model
file1 = open("CovSum.txt", "w") # writes quantitative data to output file
graph_count = 1
for item in P_H2:
    [covO] = NP_calc(npoints,T,item,kub_A,kub_B,kub_C,kub_mA,kub_mB,kub_mC,kub_bA,kub_bB,kub_bC)
    N = len(covO)
    aveO = sum(covO)/N
    file1.write(str(np.log10(item*10**(-5)))+" "+str(aveO)+" "+str(max(covO))+" "+str(min(covO))+"\n")

    covO = np.reshape(covO,(npoints,npoints))
    
    fig, ax = plt.subplots(subplot_kw=dict(projection='polar'),figsize=(9,8))
    im = ax.contourf(ph1,ph2,covO,vmin=0.00,vmax=0.98)
    ax.set_yticklabels([])
    ax.set_xticklabels([])
    plt.setp(ax.spines.values(), linewidth=1.5)
    fig.colorbar(im)
    ax.set_xticks([])
    ax.set_yticks([])
    fig.set(facecolor='white')
    fig.savefig("CovO_"+str(graph_count)+".png",dpi=300)
    plt.close(fig)
    
    graph_count = graph_count + 1
    
    print("{:.2e}".format(item)+" completed!")
file1.close()